In [ ]:
import pandas as pd
import igraph as ig
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

# ── 1. Load edge list and detect communities ──
edge_file = "edge_list_threshold_0.5.csv"
edges = pd.read_csv(edge_file)
g = ig.Graph.TupleList(edges.itertuples(index=False), directed=False, edge_attrs=["Weight"])
communities = g.community_multilevel(weights=g.es['Weight'])

# Save node list with community membership
nodes_df = pd.DataFrame({
    'Node': g.vs['name'],
    'Community': communities.membership
})
nodes_df.to_csv("node_list_with_communities.csv", index=False) #Figure_3D_data file

# ── 2. Load node list with PCN and plot top 5 communities ──
nodes_df = pd.read_csv("node_list_with_communities.csv") #Figure_3D_data file 
nodes_df['Log10_PCN'] = np.log10(nodes_df['PIRACopyNumber'])

# Top 5 communities by node count
top_5 = nodes_df['Community'].value_counts().head(5).index
mean_pcn = nodes_df[nodes_df['Community'].isin(top_5)].groupby('Community')['Log10_PCN'].mean()
sorted_communities = mean_pcn.sort_values(ascending=False)

# Plot
fig, axes = plt.subplots(nrows=len(sorted_communities), figsize=(3, len(sorted_communities)*2), sharex=True)
community_labels = ['A', 'B', 'C', 'D', 'E']

for i, community in enumerate(sorted_communities.index):
    sns.kdeplot(
        data=nodes_df[nodes_df['Community'] == community],
        x='Log10_PCN',
        fill=True,
        common_norm=False,
        alpha=0.1,
        ax=axes[i]
    )
    axes[i].text(0.05, 0.9, f'{community_labels[i]}', transform=axes[i].transAxes, fontsize=12, color='black', weight='bold')
    axes[i].set_xlabel('PCN (log10)')
    axes[i].set_ylabel('Density')

plt.tight_layout()
plt.savefig('Figure_3D.pdf', dpi=1200, format='pdf', bbox_inches='tight')
plt.show()

community_labels
